In [ ]:
lags_elegidos = [1, 7, 14, 28]
# Lista de procesos y la operación aplicada (código seguido de criterio):

# i) Imputación de variables numéricas (sales & lag features)
num_imputer = SimpleImputer(strategy='median')

num_cols = ['sales'] + [f'sales_lag_{lag}' for lag in lags_elegidos] + ['roll_mean_7','roll_mean_28']

# Imputación global como ejemplo (en producción: por grupo sería mejor)
df[num_cols] = num_imputer.fit_transform(df[num_cols])

# ii) Imputación de variables categóricas (store)
df['store'] = df['store'].cat.add_categories(['unknown']).fillna('unknown')

# iii) Codificación de variables categóricas
item_freq = df['item'].value_counts(normalize=True)
df['item_freq_enc'] = df['item'].map(item_freq)

# One-hot para store verificar si se cambia por otro tipo de categorización
top_stores = df['store'].value_counts().nlargest(10).index.tolist()
for s in top_stores:
    df[f'store_is_{s}'] = (df['store'] == s).astype(int)

# iv) Tratamiento de outliers (sales)
Q1 = df['sales'].quantile(0.25)
Q3 = df['sales'].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

df['sales_clipped'] = df['sales'].clip(lower=lower, upper=upper)

# v) Transformación de variables numéricas
df['sales_log1p'] = np.log1p(df['sales_clipped'])

# vi) Escalado de características
scale_cols = ['item_freq_enc','sales_lag_1','sales_lag_7','roll_mean_7','sales_log1p']
scaler = StandardScaler()
df[scale_cols] = scaler.fit_transform(df[scale_cols])

print('Transformaciones aplicadas. Muestra:')
print(df[scale_cols + ['sales']].head())

# Guardar dataset transformado para siguiente etapa (pipeline/operadores)
out_cols = ['date','store','item','sales'] + [f'sales_lag_{lag}' for lag in lags_elegidos] + ['roll_mean_7','roll_mean_28','item_freq_enc','sales_clipped','sales_log1p'] + [f'store_is_{s}' for s in top_stores]

df_out = df[out_cols].copy()
df_out.to_parquet('train_feature_exploration.parquet', index=False)
print('\nGuardado: train_feature_exploration.parquet con shape', df_out.shape)